In [1]:
%load_ext autoreload
%autoreload 2
import datetime
print(datetime.datetime.now())

import pygentoolbox.Tools
#dir(pygentoolbox.Tools)
import pandas as pd
import itertools
import os
import math

# TP2 = 0
# TP10 = 1
# TP16 = 2
# TP25 = 3

2019-10-22 14:28:01.383681


In [28]:
f = 'D:\\LinuxShare\\Projects\\Theresa\\Hisat2\\TotalsRNA\\Ptiwi08KD\\Pt_51_Mac\\AllTPMcounts.tsv'
cutoff = 10  # minimum read count
# f = input('Path to count matrix file please:')
#PATH: 'D:\\LinuxShare\\Projects\\Perwin\\RNA_SEQ\\Asexual_Maturation_Xy.txt'

df = pd.read_csv(f, sep='\t')
print(df.columns.values)
df = df[df['Feature'] == 'gene']  ##### IMPORTANT! FILTERING TO ONLY GENES! #####
# print(df['ID'].apply(lambda x: x.split(';')[0][3:]))
# df.index = df['Scaffold'] + '.' + df['Feature'] + '.' + df['Start'].map(str) + '.' + df['End'].map(str)  # df.index = list(df.iloc[:, 0]) # set first column as row indeces
df.index = df['ID'].apply(lambda x: x.split(';')[0][3:])
# df['ScaffoldPosition'] = df['Scaffold'] + '.' + df['Feature'] + '.' + df['Start'].map(str) + '.' + df['End'].map(str)  # df.index = list(df.iloc[:, 0]) # set first column as row indeces

dftrim = df.loc[:, ['tpm_EV.Late', 'tpm_Ptiwi08KD']]  # 'ScaffoldPosition'
print(dftrim.head())
print(dftrim.shape)

# remove 0 or negative numbers
dftrim = dftrim[(dftrim >= cutoff).all(1)]  # all values in each row need to be > 0 to be kept
# dftrim[(dftrim > 0).any(1)]  # if one value in each row is > 0 keep it
print(dftrim.shape)

# d = {df.columns.values[0] : list(dftrim.index)}
dfout = pd.DataFrame(index=dftrim.index) # use indexes from dftrim as index for output
combs = list(itertools.combinations(list(range(dftrim.shape[1])), 2))
print(combs)
for t in combs:
    print(t[0], t[1])
    fc = pygentoolbox.Tools.log2_fold_change(dftrim, t[0], t[1])  # pandas df
    # div = dftrim.iloc[:, t[0]] / dftrim.iloc[:, t[1]]
    # log2FCseries = div.apply(math.log2)
    dfout['Log2FC_%dVS%d' % (t[0], t[1])] = fc

print(dfout.head())
print(dfout.shape)
dfout_sort = dfout.sort_values(by=['Log2FC_0VS1'])
print(dfout_sort.head())
print(dfout_sort.shape)

path, file = os.path.split(f)

pygentoolbox.Tools.pandas_output_to_matrix(path, 'Log2FC_sRNA_AllPairs_Cut%d.tsv' % (cutoff), dfout, '\t')
pygentoolbox.Tools.pandas_output_to_matrix(path, 'Log2FC_sRNA_AllPairs_Cut%d.sorted.tsv' % (cutoff), dfout_sort, '\t')

['Scaffold' 'Eugene' 'Feature' 'Start' 'End' 'Dot1' 'Orientation' 'Dot2'
 'ID' 'tpm_EV.Late' 'tpm_Ptiwi08KD']
                    tpm_EV.Late  tpm_Ptiwi08KD
ID                                            
PTET.51.1.G0010001     0.000000       0.997844
PTET.51.1.G0010002     0.000000       0.900771
PTET.51.1.G0010003     0.775506       0.107615
PTET.51.1.G0010004     0.000000       0.000000
PTET.51.1.G0010005     0.000000       0.051532
(41533, 2)
(280, 2)
[(0, 1)]
0 1
                    Log2FC_0VS1
ID                             
PTET.51.1.G0010107     1.264297
PTET.51.1.G0010296     0.357406
PTET.51.1.G0010421    -1.216830
PTET.51.1.G0010500     0.874151
PTET.51.1.G0010504     1.089402
(280, 1)
                    Log2FC_0VS1
ID                             
PTET.51.1.G0490162    -8.282384
PTET.51.1.G0680113    -4.531048
PTET.51.1.G0590231    -1.981097
PTET.51.1.G0100299    -1.762995
PTET.51.1.G0320080    -1.555882
(280, 1)
Output File: D:\LinuxShare\Projects\Theresa\Hisat2\TotalsRNA\P